In [272]:
import pandas
import csv
import re
import statistics

# Data Preparation

### MSA and Major MSA population totals

In [197]:
msa = {}
with open('assignment/MSA.csv') as csvfile:
    content = csv.reader(csvfile, delimiter=',')
    firstline = True
    for row in content:
        if firstline:
            firstline = False
            continue
        # make sure zip code is appropriate length before adding to dict
        else:
            z = add_leading_zeros(row[0])
            msa[z] = {'state': row[1], 'name': row[2]}

In [118]:
# creating a dict with the MSA name as key and population as value
msa_population_totals = {}

with open('assignment/major_msa_population_totals.csv') as csvfile:
    content = csv.reader(csvfile, delimiter=',')
    for row in content:
        msa_population_totals[row[0]] = int(row[1].replace(',',''))

### Zip Code Data

The python zipcode library does not contain a complete list of zipcodes, so I downloaded a complete list from https://www.aggdata.com/node/86

In [156]:
zipcode_data = {}
with open('us_postal_codes.csv') as csvfile:
    content = csv.reader(csvfile, delimiter=',')
    firstline = True
    for row in content:
        if firstline:
            firstline = False
            continue
        else:
            zipcode = row[0]
            city = row[1]
            state = row[2]
            state_short = row[3]
            county = row[4]
            lat = row[5]
            long = row[6]
            
            zipcode_data[zipcode] = {'city': city, 'state': state, 'state_short': state_short, 
                                     'county': county, 'latitude': lat, 'longitude': long}

### Guest Days

There are cases of missing data and cases in which certain data elements are not in the correct columns in the supplied CSV. Below are a few functions I wrote to extract and sanity check data from Guest Days.csv. Ultimately I create two lists, one containing US Guest Days and the other containing all other entries (international customers, US rows that either don't have a zip code at all or the zip code isn't valid) is saved to a separate list for further cleaning.

In [233]:
def clean_guestdays(row):
### takes in a single row from the guest_days csv and returns zipcode, ski_days, and booking_window

    # check if the zip code is a valid US zip code
    valid_zip = False
    z = get_zipcode(row)
    
    if z in zipcode_data:
        valid_zip = True
    
    # ski days and booking window
    booking_window = None
    ski_days = row[-1]
    if ski_days:
        booking_window = row[-2] # if ski_days is present, booking_window is the second to last value in the row
    else:
        ski_days = row[-2] # otherwise, assume booking_window is missing
    
    if valid_zip:
        return True, [z, ski_days, booking_window]
    else:
        return False, row

In [234]:
def get_zipcode(row):
### Parses a row from guest_days to extract the best candidate for a zipcode
    match = re.search('(\d{5})([- ])?(\d{4})?', ' '.join(row))
    
    if match:
        # convert to string
        match = match.group(1)
    else:
        # do more work
        possible_zipcode = None
        minimum_zipcode = 210 # lowest zipcode in the US
        
        for i in row:
            # the regex expression should've caught any 5-digit or 5-plus-4 digit zip codes
            # a non-match is either not present, not a US zip, or is 3 or 4 digits
            digits = len(i)
            if 2 < digits < 5:
                try:
                    possible_zipcode = int(i)
                except ValueError:
                    pass

            if possible_zipcode:
                if possible_zipcode >= minimum_zipcode and possible_zipcode != 1000:
                    # there's a single false positive w/ zipcode 1000
                    match = str(possible_zipcode)
                    match = add_leading_zeros(match)
    
    return match

In [235]:
def add_leading_zeros(z):
### Adds appropriate amount of leading zeros to zipcode
    zeroes = {1: 4, 2: 3, 3: 2, 4: 1, 5: 0}
    return zeroes[len(z)] * '0' + z

In [236]:
guest_days_clean = []
guest_days_dirty = []

with open('assignment/guest_days.csv', newline='') as gd:
    reader = csv.reader(gd, delimiter=',')
    for row in reader:
        result = clean_guestdays(row)
        if result[0]:
            guest_days_clean.append(result[1])
        else:
            guest_days_dirty.append(result[1])

## Check data loss

I filtered the data, so let's see how much was excluded and browse the excluded data

In [237]:
loss = 1 - len(guest_days_clean) / float(len(guest_days_clean) + len(guest_days_dirty))

In [238]:
loss * 100 # percentage of total rows dropped

6.825587377808262

In [434]:
guest_days_dirty = guest_days_dirty[1:] # dropping the header row

Browsing the "dirty" guest days, we can see that most of them are international, though some are in the United States but were dropped because a valid zip code was not provided.

Now, let's see the total percentage of guest days dropped

In [240]:
total_clean_guestdays = 0
total_dirty_guestdays = 0

for row in guest_days_clean:
    total_clean_guestdays += int(row[1])

for row in guest_days_dirty:
    # guest_days is either the last or second to last entry in each row
    try:
        guest_days = int(row[-1])
    except ValueError:
        guest_days = int(row[-2])
    
    total_dirty_guestdays += guest_days

In [241]:
total_clean_guestdays

27422

In [242]:
total_dirty_guestdays

2075

In [243]:
loss_guestdays = 1 - total_clean_guestdays / float(total_clean_guestdays + total_dirty_guestdays)
loss_guestdays * 100

7.034613689527747

### Create DataFrames

Since there are often multiple rows per zip code, and differing values of "booking_window" across rows with the same zip code, we'll need to summarize booking window. Complicating this is the "15+ Days" value that can occur in booking_window. Since the other possible values are either discrete ("02 Days") or a range, I'll aggregate the data and find an approximate median for booking_window. The values that are ranges run from `x` to `2x - 1`, so I'm taking the liberty of assuming that "15+ Days" is actually a range of "15-29 Days". I ended up being unable to use the booking window data because I found out later that it was missing from enough rows to where I felt its use would be too imprecise. 

In [280]:
booking_window_map = {'00 Same Day': 0,
                      '01 Day': 1, 
                      '02 Days': 2, 
                      '03 Days': 3, 
                      '04-7 Days': statistics.median([4,5,6,7]), 
                      '08-14 Days': statistics.median([8,9,10,11,12,13,14]),
                      '15+ Days': statistics.median([15,16,17,18,19,20,21,22,23,24,25,26,27,28,29])}

In [281]:
booking_window_map

{'00 Same Day': 0,
 '01 Day': 1,
 '02 Days': 2,
 '03 Days': 3,
 '04-7 Days': 5.5,
 '08-14 Days': 11,
 '15+ Days': 22}

Now, I'll rebuid guest_days_clean to only contain a single entry per zip code before importing it in to pandas

In [300]:
guest_days_clean_unique = {}
for row in guest_days_clean:
    # If booking window is present, use it, otherwise don't
    if row[2]:
        booking_window = booking_window_map[row[2]]
    else:
        booking_window = 0
        
    if row[0] not in guest_days_clean_unique:
        guest_days_clean_unique[row[0]] = {'guest_days': float(row[1]), 'avg_booking_window': booking_window * float(row[1])}
    else:
        guest_days_clean_unique[row[0]]['guest_days'] += float(row[1])
        guest_days_clean_unique[row[0]]['avg_booking_window'] += booking_window * float(row[1])

Now I'll adjust the booking_window values to be averaged by the number of guest_days per zip

In [302]:
for z in guest_days_clean_unique:
    guest_days_clean_unique[z]['avg_booking_window'] = guest_days_clean_unique[z]['avg_booking_window'] / guest_days_clean_unique[z]['guest_days']

Flatten guest_days_clean_uniqe to import into pandas

In [315]:
guest_days = []
for z in guest_days_clean_unique:
    guest_days.append([z,  guest_days_clean_unique[z]['guest_days'], guest_days_clean_unique[z]['avg_booking_window']])

In [317]:
guest_days = pandas.DataFrame(data=guest_days, columns=['zip_code', 'guest_days', 'avg_booking_window'])

Now I'll incorporate the data for each zip code

In [319]:
def get_zipcode_data(z):
    zd = zipcode_data[z]
    return zd

def get_city(zd):
    return zd['city']

def get_state(zd):
    return zd['state']

def get_state_short(zd):
    return zd['state_short']

def get_county(zd):
    return zd['county']

def get_latitude(zd):
    return zd['latitude']

def get_longitude(zd):
    return zd['longitude']

In [320]:
guest_days['city'] = guest_days['zip_code'].apply(get_zipcode_data).apply(get_city)
guest_days['state'] = guest_days['zip_code'].apply(get_zipcode_data).apply(get_state)
guest_days['state_short'] = guest_days['zip_code'].apply(get_zipcode_data).apply(get_state_short)
guest_days['county'] = guest_days['zip_code'].apply(get_zipcode_data).apply(get_county)
guest_days['latitude'] = guest_days['zip_code'].apply(get_zipcode_data).apply(get_latitude)
guest_days['longitude'] = guest_days['zip_code'].apply(get_zipcode_data).apply(get_longitude)

In [438]:
guest_days.head()

,zip_code,guest_days,avg_booking_window,city,state,state_short,county,latitude,longitude,msa,population
0,01033,6.0,22.0,Granby,Massachusetts,MA,Hampshire,42.2557,-72.52,"Springfield, MA MSA",6227
1,01057,1.0,22.0,Monson,Massachusetts,MA,Hampden,42.101,-72.3196,"Springfield, MA MSA",8534
2,01060,3.0,11.0,Northampton,Massachusetts,MA,Hampshire,42.3223,-72.6313,"Springfield, MA MSA",15284
3,01082,1.0,22.0,Ware,Massachusetts,MA,Hampshire,42.2618,-72.2583,"Springfield, MA MSA",10322
4,01085,1.0,22.0,Westfield,Massachusetts,MA,Hampden,42.1251,-72.7495,"Springfield, MA MSA",41117


# Analysis

Incorporating MSA statistics

In [335]:
def get_msa(z):
    return msa[z]['name']

guest_days['msa'] = guest_days['zip_code'].apply(get_msa)

In [439]:
guest_days.head()

,zip_code,guest_days,avg_booking_window,city,state,state_short,county,latitude,longitude,msa,population
0,01033,6.0,22.0,Granby,Massachusetts,MA,Hampshire,42.2557,-72.52,"Springfield, MA MSA",6227
1,01057,1.0,22.0,Monson,Massachusetts,MA,Hampden,42.101,-72.3196,"Springfield, MA MSA",8534
2,01060,3.0,11.0,Northampton,Massachusetts,MA,Hampshire,42.3223,-72.6313,"Springfield, MA MSA",15284
3,01082,1.0,22.0,Ware,Massachusetts,MA,Hampshire,42.2618,-72.2583,"Springfield, MA MSA",10322
4,01085,1.0,22.0,Westfield,Massachusetts,MA,Hampden,42.1251,-72.7495,"Springfield, MA MSA",41117


Unfortunately, the major msa population totals provided don't cover all of the MSA's in the guest days dataset. That's okay, though, because I have the internet.

In [339]:
pop = {}
with open('2010_census_population.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    first_row = True
    for row in reader:
        if first_row:
            first_row = False
            continue
        else:
            pop[row[0]] = row[1]
            

Add population data to dataframe, and then weight guest_days by population

In [350]:
def get_population(z):
    if z in pop:
        return pop[z]
    else:
        return None
    
guest_days['population'] = guest_days['zip_code'].apply(get_population)

In [346]:
matches = 0
for p in guest_days['population']:
    if p:
        matches += 1

Export data to CSV to generate a heatmap

In [353]:
len(guest_days) - matches

79

79 zip codes did not match the ZCTA codes provided by Census, dropping them

In [359]:
gd = guest_days.dropna(subset=['population'])

What zip codes have the best market penetration (guest days / population)?

In [363]:
gd['population'] = pandas.to_numeric(gd['population'], errors='coerce')

C:\Anaconda\envs\py36\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [368]:
gd['market_penetration'] = gd['guest_days'] / gd['population']

C:\Anaconda\envs\py36\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [440]:
gd.head()

,zip_code,guest_days,avg_booking_window,city,state,state_short,county,latitude,longitude,msa,population,market_penetration,mp_scaled
0,01033,6.0,22.0,Granby,Massachusetts,MA,Hampshire,42.2557,-72.52,"Springfield, MA MSA",6227,0.000964,0.004125
1,01057,1.0,22.0,Monson,Massachusetts,MA,Hampden,42.101,-72.3196,"Springfield, MA MSA",8534,0.000117,0.000458
2,01060,3.0,11.0,Northampton,Massachusetts,MA,Hampshire,42.3223,-72.6313,"Springfield, MA MSA",15284,0.000196,0.000800
3,01082,1.0,22.0,Ware,Massachusetts,MA,Hampshire,42.2618,-72.2583,"Springfield, MA MSA",10322,0.000097,0.000370
4,01085,1.0,22.0,Westfield,Massachusetts,MA,Hampden,42.1251,-72.7495,"Springfield, MA MSA",41117,0.000024,0.000055


Export to csv to create a heatmap

In [406]:
gd = gd.replace([np.inf, -np.inf], np.nan)
gd = gd.dropna(subset=['market_penetration'])
gd_trunc = gd
drop_these = ['zip_code', 'guest_days', 'avg_booking_window', 'city', 'state', 'state_short', 'county', 'msa', 'population']
for d in drop_these:
    gd_trunc = gd_trunc.drop(d, 1)

In [441]:
gd_trunc.head()

,lat,lon,market_penetration,mp_scaled
0,42.2557,-72.52,0.000964,0.004125
1,42.101,-72.3196,0.000117,0.000458
2,42.3223,-72.6313,0.000196,0.000800
3,42.2618,-72.2583,0.000097,0.000370
4,42.1251,-72.7495,0.000024,0.000055


In [408]:
gd_trunc.columns = ['lat', 'lon', 'market_penetration']

In [410]:
# normalize the market penetration data for easier viewing
from sklearn import preprocessing

x = gd_trunc['market_penetration'].reshape(-1,1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
gd_trunc['mp_scaled'] = x_scaled

C:\Anaconda\envs\py36\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [442]:
gd_trunc.head()

,lat,lon,market_penetration,mp_scaled
0,42.2557,-72.52,0.000964,0.004125
1,42.101,-72.3196,0.000117,0.000458
2,42.3223,-72.6313,0.000196,0.000800
3,42.2618,-72.2583,0.000097,0.000370
4,42.1251,-72.7495,0.000024,0.000055


In [412]:
with open('gd.csv', 'w') as f:
    gd_trunc.to_csv(f)

In [413]:
gd_trunc2 = gd

In [415]:
state_guest_days = gd_trunc2.groupby('state_short')['guest_days'].sum()
state_populations = gd_trunc2.groupby('state_short')['population'].sum()

In [419]:
states = state_guest_days.keys()

In [423]:
state_penetration = pandas.concat([state_guest_days, state_populations], axis=1)

In [425]:
state_penetration['mp'] = state_penetration['guest_days'] / state_penetration['population']

In [428]:
state_penetration = state_penetration.drop('guest_days', 1)
state_penetration = state_penetration.drop('population', 1)

In [429]:
state_penetration

,mp
state_short,
AK,0.000095
AL,0.000389
AR,0.000432
AZ,0.000208
CA,0.000291
CO,0.000150
CT,0.000332
DC,0.000484
DE,0.000162


In [430]:
x = state_penetration['mp'].reshape(-1,1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
state_penetration['mp_scaled'] = x_scaled

C:\Anaconda\envs\py36\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


In [432]:
state_penetration = state_penetration.drop('mp', 1)

In [433]:
with open('sp.csv', 'w') as f:
    state_penetration.to_csv(f)